In [13]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
import csv
import aot

base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]
video_db_path = base_dir / "data/videos/database_originals.tsv"
video_db = pd.read_csv(video_db_path, sep="\t")

main_settings_dir = (
    base_dir / core_settings["paths"]["settings_path"] / "main"
)
memory_settings_dir = (
    base_dir / core_settings["paths"]["settings_path"] / "memory_main"
)
subject_number = core_settings["various"]["subject_number"]
run_number = core_settings["various"]["run_number"]
session_number = core_settings["various"]["session_number"]
unique_video_number_per_session = core_settings["various"][
    "unique_video_number_per_session"
]
img_number_per_session = unique_video_number_per_session

In [14]:
def answer_generate(subject, session):
    print(f"subject: {subject}, session: {session}")
    # read main experiment settings
    main_runs_settings = []
    main_runs_videos = []
    main_runs_videos_name = []  # raw video name without .mp4

    memory_runs_settings = []
    memory_runs_pictures = []
    memory_runs_pictures_name = []  # raw video name without .mp4

    answer_dict = {}  # picture_name:Y/N

    for ind in range(run_number):
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = main_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        main_runs_settings.append(runsetting)
    for runsetting in main_runs_settings:
        raw_video_list = runsetting["stimuli"]["movie_files"]
        video_list = [video for video in raw_video_list if video != "blank"]
        # print(video_list)
        main_runs_videos.append(video_list)
    for videos in main_runs_videos:
        name_list = [video.split(".")[0] for video in videos]
        main_runs_videos_name.append(name_list)

    flat_main_runs_videos_name = [
        item for sublist in main_runs_videos_name for item in sublist
    ]

    flat_main_runs_videos_name = [name.replace("_rv", "_fw") for name in flat_main_runs_videos_name]

    for ind in [0]:# only one run for each memory session
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = memory_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        memory_runs_settings.append(runsetting)
    for runsetting in memory_runs_settings:
        raw_picture_list = runsetting["stimuli"]["picture_files"]
        picture_list = [picture for picture in raw_picture_list if picture != "blank"]
        # print(video_list)
        memory_runs_pictures.append(picture_list)
    for pictures in memory_runs_pictures:
        name_list = [picture.split(".")[0] for picture in pictures]
        memory_runs_pictures_name.append(name_list)

    flat_memory_runs_pictures_name = [
        item for sublist in memory_runs_pictures_name for item in sublist
    ]

    for picture_name in flat_memory_runs_pictures_name:
        if picture_name in flat_main_runs_videos_name:
            answer_dict[picture_name] = "Y"
        else:
            answer_dict[picture_name] = "N"

    # save answer_dict to tsv file
    answer_dict_file_path = memory_settings_dir / "answers" / f"answer_dict_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}.tsv"
    with open(answer_dict_file_path, "w") as f:
        writer = csv.writer(f, delimiter="\t")
        for key, value in answer_dict.items():
            writer.writerow([key, value])
    
    print("answer dict for subject {subject} session {session}:",answer_dict)
    return answer_dict

In [15]:
def generate_answers_for_all_subjects():
    for subject in range(1,subject_number+1):
        for session in range(1,session_number+1):
            answer_generate(subject, session)   

In [16]:
generate_answers_for_all_subjects()


subject: 1, session: 1


FileNotFoundError: [Errno 2] No such file or directory: '/tank/shared/2022/arrow_of_time/arrow_of_time_exp/aot/data/experiment/settings/memory_main/answers/answer_dict_sub_01_ses_01.tsv'